In [1]:
import pandas as pd
import numpy as np
import datetime as dt #
import seaborn as sns
import matplotlib.pyplot as plt

 
from pygam import LinearGAM, s, f #
import math #
from scipy import stats
from sklearn.metrics import mean_squared_error #

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)



In [2]:
log_var_df = pd.read_csv('../log_all_variables.csv').drop(columns='Unnamed: 0') # replace with file path
log_var_df.date = pd.to_datetime(log_var_df.date)
log_var_df = log_var_df[~log_var_df.TARGET1.isna() & ~log_var_df.TARGET2.isna()] # drop forecasts too far out
log_var_df

,date,city_name,ww_raw,hosp_raw,WW_lag1,WW_lag2,WW_lag3,Hosp_lag1,Hosp_lag2,Hosp_lag3,...,theme2_minority_status_and_language,theme3_housing_type_and _transport,theme4_epidemiological_factors,theme5_healthcare_system,theme6_high_risk_environments,theme7_population_density,ccvi_std,TARGET0,TARGET1,TARGET2
0,2021-01-08,Houston,14.000239,7.146210,13.940836,13.848086,13.749523,6.985642,6.816736,6.676454,...,0.929376,0.465482,0.173536,0.766310,0.719819,0.897120,0.174664,7.146210,7.230149,7.199038
1,2021-01-08,San Francisco,16.435822,5.417581,16.067153,16.200074,15.770062,5.291628,5.145555,4.988162,...,0.891312,0.339281,0.115164,0.334438,0.221934,0.985472,0.220888,5.417581,5.482027,5.427883
2,2021-01-08,Denver,11.492996,5.507072,11.932021,12.096300,12.291047,5.650482,5.738874,5.888482,...,0.770047,0.317233,0.059432,0.260693,0.676903,0.907949,0.216895,5.507072,5.456236,5.419270
3,2021-01-15,Denver,11.232900,5.456236,11.492996,11.932021,12.096300,5.507072,5.650482,5.738874,...,0.770047,0.317233,0.059432,0.260693,0.676903,0.907949,0.216895,5.456236,5.419270,5.231109
4,2021-01-15,San Francisco,16.434781,5.482027,16.435822,16.067153,16.200074,5.417581,5.291628,5.145555,...,0.891312,0.339281,0.115164,0.334438,0.221934,0.985472,0.220888,5.482027,5.427883,5.316485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2022-09-09,San Francisco,16.197034,4.330733,16.268709,16.352845,16.445698,4.426841,4.601831,4.743482,...,0.891312,0.339281,0.115164,0.334438,0.221934,0.985472,0.220888,4.330733,4.267364,4.174387
472,2022-09-09,NYC,15.519371,7.827573,15.618614,15.732276,15.850216,7.858963,7.899834,7.929187,...,0.900737,0.533193,0.294657,0.845080,0.129549,0.988377,0.237060,7.827573,7.804455,7.813996
473,2022-09-16,San Diego,15.541750,4.491575,15.666943,15.737539,15.864026,4.644391,4.710816,4.873014,...,0.858132,0.291705,0.087841,0.748704,0.397800,0.931668,0.242273,4.491575,4.482195,4.559874
474,2022-09-16,San Francisco,16.063946,4.267364,16.197034,16.268709,16.352845,4.330733,4.426841,4.601831,...,0.891312,0.339281,0.115164,0.334438,0.221934,0.985472,0.220888,4.267364,4.174387,4.060443


In [3]:
testwks=83
train_min = log_var_df.date.unique()[6]


# Out of a total 89 weeks in the study period, 
# the inital training set is made up of the first 6 weeks (25 datapoints)
# The testing set is made up of the final 83 weeks 

In [4]:
#creating dataframe to hold predictions using data from the input file

log_pred_df = pd.DataFrame(log_var_df[['date', 'city_name', 'TARGET0', 'TARGET1', 'TARGET2']])
log_pred_df.columns= ['date', 'city_name', 'groundtruth1', 'groundtruth2', 'groundtruth3']
log_pred_df[['1weekPred', '2weekPred', '3weekPred']] = np.nan

log_pred_df.head(10)

,date,city_name,groundtruth1,groundtruth2,groundtruth3,1weekPred,2weekPred,3weekPred
0,2021-01-08,Houston,7.146210,7.230149,7.199038,NaN,NaN,NaN
1,2021-01-08,San Francisco,5.417581,5.482027,5.427883,NaN,NaN,NaN
2,2021-01-08,Denver,5.507072,5.456236,5.419270,NaN,NaN,NaN
3,2021-01-15,Denver,5.456236,5.419270,5.231109,NaN,NaN,NaN
4,2021-01-15,San Francisco,5.482027,5.427883,5.316485,NaN,NaN,NaN
5,2021-01-15,Houston,7.230149,7.199038,7.125858,NaN,NaN,NaN
6,2021-01-22,Denver,5.419270,5.231109,5.139154,NaN,NaN,NaN
7,2021-01-22,Houston,7.199038,7.125858,7.077619,NaN,NaN,NaN
8,2021-01-22,San Francisco,5.427883,5.316485,5.156178,NaN,NaN,NaN
9,2021-01-29,Denver,5.231109,5.139154,5.046186,NaN,NaN,NaN


In [5]:
def generate_CI_sample(model, X_new, ci_df):
    
    # Generates 50 quantile upper and lower bound samples
    #
    # model: the updated model for that week of study
    # X_new: predictions just made for the forecasting window
    # ci_df: file to save the quantile samples
    
    
    for c in np.linspace(.01,0.99, 50): 

        # generate GAM prediction interval of that quantile
        pred_intervals = model.prediction_intervals(X_new, width = c) 
        
        # save confidence intervals to quantile sample file
        ci_df.loc[X_new.index, [f"CI_{round(c,2)}_lo",f"CI_{round(c,2)}_hi"]] = pred_intervals

    return ci_df
    

In [6]:
# Create confidence interval files for each forecasting window
RandSamp0 = log_pred_df[['date', 'city_name']]
RandSamp1 = log_pred_df[['date','city_name']]
RandSamp2 = log_pred_df[['date','city_name']]


for d in range(testwks-1):
    # for prediction point, make a training set equivalent to the 
    # training set plus the weeks prior to the prediction
 
    weeks = log_var_df[log_var_df.date <= train_min+np.timedelta64(d, 'W')]
    
    X_train = weeks.iloc[:, 4:-3] #training data is made up of all input variables
    Y0_train = weeks.iloc[:, -3] # 1 week out true value
    Y1_train = weeks.iloc[:, -2] # 2 week out true value
    Y2_train = weeks.iloc[:, -1] # 3 week out true value
    

    n_splines = 4 # non-linearity parameter

    lambd = .05 # smoothing parameter
    
    
    #  ONE WEEK PREDICTION - generate a model each week to do 1 week out predictions 
    
    gam0 = LinearGAM(s(0,n_splines=n_splines) + s(1,n_splines=n_splines) + s(2,n_splines=n_splines)
                + s(3,n_splines=n_splines) + s(4,n_splines=n_splines) + s(5,n_splines=n_splines)
                + s(6,n_splines=n_splines) + s(7,n_splines=n_splines)+ s(8,n_splines=n_splines)
                + s(9,n_splines=n_splines) + s(10,n_splines=n_splines) + s(11,n_splines=n_splines) 
                + s(12,n_splines=n_splines)+ s(13,n_splines=n_splines) + s(14,n_splines=n_splines) 
                + s(15,n_splines=n_splines)+ s(16,n_splines=n_splines) 
                + s(17,n_splines=n_splines),lam = lambd).fit(
                X_train.to_numpy(), 
                Y0_train.to_numpy())


    preds0_df = log_var_df[log_var_df.date == train_min+np.timedelta64(d+1, 'W')].iloc[:,4:-3]
        # dataframe made up of just the inputs for the testing set   

    preds0_results = gam0.predict(preds0_df)
        # predict the next set of values
        
    log_pred_df.loc[preds0_df.index, '1weekPred'] = preds0_results
        # update the prediction dataframe with the new predictions
    
    RandSamp0 = generate_CI_sample(gam0, preds0_df, RandSamp0)
        # update the confidence interval dataframe with new predictions
    
    
    #  TWO WEEK PREDICTION - generate a model each week to do 2 week out predictions
    
    gam1 = LinearGAM(s(0,n_splines=n_splines) + s(1,n_splines=n_splines) + s(2,n_splines=n_splines)
                + s(3,n_splines=n_splines) + s(4,n_splines=n_splines) + s(5,n_splines=n_splines)
                + s(6,n_splines=n_splines) + s(7,n_splines=n_splines)+ s(8,n_splines=n_splines)
                + s(9,n_splines=n_splines) + s(10,n_splines=n_splines) + s(11,n_splines=n_splines) + s(12,n_splines=n_splines)
                + s(13,n_splines=n_splines) + s(14,n_splines=n_splines) + s(15,n_splines=n_splines)
                + s(16,n_splines=n_splines) + s(17,n_splines=n_splines),lam = lambd).fit(
                X_train.to_numpy(), 
                Y1_train.to_numpy())

    preds1_df = log_var_df[log_var_df.date == train_min+np.timedelta64(d+1, 'W')].iloc[:,4:-3]
        # dataframe made up of just the inputs for the testing set 
        
    preds1_results = gam1.predict(preds1_df)
        # predict the next set of values
    
    log_pred_df.loc[preds1_df.index, '2weekPred'] = preds1_results
        # update the prediction dataframe with the new predictions

    RandSamp1 = generate_CI_sample(gam1, preds1_df, RandSamp1)
     # update the confidence interval dataframe with new predictions
    
    
    # THREE WEEK PREDICTION - generate a model each week to do 3 week out predictions
    
    gam2 = LinearGAM(s(0,n_splines=n_splines) + s(1,n_splines=n_splines) + s(2,n_splines=n_splines)
                + s(3,n_splines=n_splines) + s(4,n_splines=n_splines) + s(5,n_splines=n_splines)
                + s(6,n_splines=n_splines) + s(7,n_splines=n_splines)+ s(8,n_splines=n_splines)
                + s(9,n_splines=n_splines) + s(10,n_splines=n_splines) + s(11,n_splines=n_splines) + s(12,n_splines=n_splines)
                + s(13,n_splines=n_splines) + s(14,n_splines=n_splines) + s(15,n_splines=n_splines)
                + s(16,n_splines=n_splines) + s(17,n_splines=n_splines),lam = lambd).fit(
                X_train.to_numpy(), 
                Y2_train.to_numpy())

    preds2_df = log_var_df[log_var_df.date == train_min+np.timedelta64(d+1, 'W')].iloc[:,4:-3]
        # dataframe made up of just the inputs for the testing set 
        
    preds2_results = gam2.predict(preds2_df)
        # predict the next set of values
    
    log_pred_df.loc[preds2_df.index, '3weekPred'] = preds2_results
        # update the prediction dataframe with the new predictions
    
    RandSamp2 = generate_CI_sample(gam2, preds2_df, RandSamp2)
     # update the confidence interval dataframe with new predictions
        

# Categorization Models

In [7]:
# exponentiate all random samples and predictions to categorize correctly

for r in RandSamp1.columns[2:]:
    RandSamp0[f'{r}'] = (np.exp(RandSamp0[f"{r}"]))
    RandSamp1[f'{r}'] = (np.exp(RandSamp1[f"{r}"]))
    RandSamp2[f'{r}'] = (np.exp(RandSamp2[f"{r}"]))


for r in log_pred_df.columns[2:]:

    log_pred_df[f'{r}'] = (np.exp(log_pred_df[f"{r}"]))



/var/folders/1q/ndf5jz855p7g31_k_pss1nl40000gn/T/ipykernel_59565/938015013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RandSamp0[f'{r}'] = (np.exp(RandSamp0[f"{r}"]))
/var/folders/1q/ndf5jz855p7g31_k_pss1nl40000gn/T/ipykernel_59565/938015013.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RandSamp1[f'{r}'] = (np.exp(RandSamp1[f"{r}"]))
/var/folders/1q/ndf5jz855p7g31_k_pss1nl40000gn/T/ipykernel_59565/938015013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

## Hospital Capacity Risk Categorization (HCR)

In [8]:
cities = [ 'Charlotte', 'Houston', 'NYC', 'Denver', 'San Diego', 'San Francisco']

# Average number of available beds for each city over the study period
gen_avg = {}
gen_avg['Houston']= 9500            
gen_avg['NYC']= 55000                
gen_avg['Denver'] = 3300             
gen_avg['San Diego'] = 3700          
gen_avg['San Francisco'] = 11900     
gen_avg['Charlotte'] = 3200          

In [9]:
def HCR_categorize(values, all_threshold, predweek):
    
    # Converts continous predictions into HCR categorizations
    #
    # values: continous predictions
    # all_threshold: hospital capacity
    # predweek: forecasting window
    
    prev = values.shift(predweek) 
    cats = []
    
    
    for value in prev:
        # saves the category based on the percent of beds used for covid hospitalizatoins 
        
        if np.isnan(value): 
            cats.append(np.nan)
        elif value < 0.02*all_threshold: 
            cats.append(1)
        elif value < 0.05*all_threshold:
            cats.append(2)
        elif value < 0.075*all_threshold:
            cats.append(3)
        elif value < 0.1*all_threshold:
            cats.append(4)
        else:
            cats.append(5)
            
    return cats


In [10]:
# Converts numerical categories into name of level 
translate_HCR = {}
translate_HCR['date'] = "date"
translate_HCR['MLE_category'] = "MLE_category"
translate_HCR[5.0] = "very_high"
translate_HCR[4.0] = "high"
translate_HCR[3.0] = 'moderate'
translate_HCR[2.0] = "low"
translate_HCR[1.0] = "very_low"


def rs_HCR_categorize(randomsamples, threshold, predweek):
    
    # generates the maximum likelihood of the categorical prediction 
    #
    # randomsamples: dataframe of confidence interval samples
    # threshold: hospital capacity
    # predweek: forecasting window 
    
    
    randsampcat = randomsamples[['date']].copy()

    for j in randomsamples.iloc[:,2:]:

        randsampcat[f'{j}'] = HCR_categorize(randomsamples[f"{j}"], threshold, predweek)

    randsampcat = randsampcat.set_index("date")

    randsampcat = randsampcat.apply(pd.Series.value_counts, axis=1).fillna(0).reset_index()
    randsampcat.index = thiscitysamp.index
    randsampcat['MLE_category'] = randsampcat.iloc[:,1:].idxmax(axis=1)
    randsampcat.columns = [translate_HCR[c] for c in randsampcat.columns]
    randsampcat=randsampcat.fillna(0)
    
    return randsampcat

### 1 Week  Forecast

In [12]:
catone_hcr = log_pred_df[['date','city_name', 'groundtruth1','1weekPred']].copy() # selecting for 1 week predictions

catone_hcr.columns=['date', 'city_name', 'true_hosp', 'forecast_hosp']
catone_hcr[["truth_category", "pred_category"]] = np.nan
catone_hcr[['very_low', 'low', 'moderate', 'high', 'very_high']] = np.nan

for city in cities: # categorize cities individually 
    
    thiscity = catone_hcr[catone_hcr.city_name == city] # subset city
    thiscitysamp = RandSamp0.loc[thiscity.index] # subset samples for city

    truthcat_array = HCR_categorize(thiscity.true_hosp, gen_avg[f"{city}"], 1) # categorize ground truth
    predcat_array = HCR_categorize(thiscity.forecast_hosp, gen_avg[f"{city}"], 1) # categorize raw predictions
    
    # save categorizations
    catone_hcr.loc[thiscity.index, "truth_category"] = truthcat_array 
    catone_hcr.loc[thiscity.index, "pred_category"] = predcat_array


    RS_cat1 = rs_HCR_categorize(thiscitysamp, gen_avg[f"{city}"], 1) # categorize random samples
    
    for category in RS_cat1.columns[1:]:
        catone_hcr.loc[thiscitysamp.index, f'{category}'] = RS_cat1[f'{category}'] # add sample lables and maximum likelihood 
        

catone_hcr= catone_hcr[catone_hcr.pred_category.notnull()].fillna(0) # remove training period from 
catone_hcr.head()


,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
31,2021-03-05,Houston,796.857143,1033.782303,4.0,5.0,0.0,0.0,0.0,9.0,91.0,5.0
32,2021-03-05,San Francisco,79.500000,116.962832,1.0,1.0,98.0,2.0,0.0,0.0,0.0,1.0
33,2021-03-05,Denver,126.428571,148.767082,2.0,2.0,0.0,69.0,30.0,1.0,0.0,2.0
34,2021-03-05,NYC,6042.000000,6788.576534,5.0,5.0,0.0,0.0,0.0,4.0,96.0,5.0
35,2021-03-05,Charlotte,199.333333,226.822857,3.0,4.0,0.0,0.0,21.0,54.0,25.0,4.0


### Two Week Out Forecast

In [13]:
cattwo_hcr = log_pred_df[['date','city_name', 'groundtruth2','2weekPred']].copy() # selecting for 2 week predictions

cattwo_hcr.columns=['date', 'city_name', 'true_hosp', 'forecast_hosp']
cattwo_hcr[["truth_category", "pred_category"]] = np.nan
cattwo_hcr[['very_low', 'low', 'moderate', 'high', 'very_high']] = np.nan

for city in cities: # categorize cities individually 
    
    thiscity = cattwo_hcr[cattwo_hcr.city_name == city] # subset city
    thiscitysamp = RandSamp1.loc[thiscity.index] # subset samples for city

    truthcat_array = HCR_categorize(thiscity.true_hosp, gen_avg[f"{city}"], 2) # categorize ground truth
    predcat_array = HCR_categorize(thiscity.forecast_hosp, gen_avg[f"{city}"], 2) # categorize raw predictions
    
    # save categorizations
    cattwo_hcr.loc[thiscity.index, "truth_category"] = truthcat_array 
    cattwo_hcr.loc[thiscity.index, "pred_category"] = predcat_array


    RS_cat2 = rs_HCR_categorize(thiscitysamp, gen_avg[f"{city}"], 2) # categorize random samples
    
    for category in RS_cat2.columns[1:]:
        cattwo_hcr.loc[thiscitysamp.index, f'{category}'] = RS_cat2[f'{category}'] # add sample lables and maximum likelihood 
        

cattwo_hcr = cattwo_hcr[cattwo_hcr.pred_category.notnull()].fillna(0) # remove training period from 
cattwo_hcr

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
36,2021-03-12,Houston,724.285714,811.598527,4.0,5.0,0.0,0.0,2.0,22.0,76.0,5.0
37,2021-03-12,NYC,5414.666667,5131.355208,5.0,5.0,0.0,0.0,2.0,16.0,82.0,5.0
38,2021-03-12,San Francisco,51.500000,64.839049,1.0,1.0,99.0,1.0,0.0,0.0,0.0,1.0
39,2021-03-12,Charlotte,149.666667,151.122482,3.0,4.0,0.0,4.0,44.0,40.0,12.0,3.0
40,2021-03-12,Denver,129.857143,66.690957,2.0,2.0,0.0,71.0,28.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
471,2022-09-09,San Francisco,71.333333,63.962725,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0
472,2022-09-09,NYC,2451.500000,2444.049257,2.0,2.0,0.0,55.0,44.0,1.0,0.0,2.0
473,2022-09-16,San Diego,88.428571,96.389235,2.0,2.0,1.0,99.0,0.0,0.0,0.0,2.0
474,2022-09-16,San Francisco,65.000000,63.867785,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0


### Three Week Out Forecast

In [14]:
cattre_hcr = log_pred_df[['date','city_name', 'groundtruth3','3weekPred']].copy() # selecting for 1 week predictions

cattre_hcr.columns=['date', 'city_name', 'true_hosp', 'forecast_hosp']
cattre_hcr[["truth_category", "pred_category"]] = np.nan
cattre_hcr[['very_low', 'low', 'moderate', 'high', 'very_high']] = np.nan

for city in cities: # categorize cities individually 
    
    thiscity = cattre_hcr[cattre_hcr.city_name == city] # subset city
    thiscitysamp = RandSamp2.loc[thiscity.index] # subset samples for city

    truthcat_array = HCR_categorize(thiscity.true_hosp, gen_avg[f"{city}"], 3) # categorize ground truth
    predcat_array = HCR_categorize(thiscity.forecast_hosp, gen_avg[f"{city}"], 3) # categorize raw predictions
    
    # save categorizations
    cattre_hcr.loc[thiscity.index, "truth_category"] = truthcat_array 
    cattre_hcr.loc[thiscity.index, "pred_category"] = predcat_array


    RS_cat3 = rs_HCR_categorize(thiscitysamp, gen_avg[f"{city}"], 3) # categorize random samples
    
    for category in RS_cat3.columns[1:]:
        cattre_hcr.loc[thiscitysamp.index, f'{category}'] = RS_cat3[f'{category}'] # add sample lables and maximum likelihood 
        

cattre_hcr = cattre_hcr[cattre_hcr.pred_category.notnull()].fillna(0) # remove training period from 
cattre_hcr

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
41,2021-03-19,NYC,5246.166667,4779.088045,5.0,5.0,0.0,0.0,7.0,28.0,65.0,5.0
42,2021-03-19,San Francisco,38.000000,47.470068,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0
43,2021-03-19,Houston,616.714286,599.986608,4.0,5.0,0.0,0.0,7.0,33.0,60.0,5.0
44,2021-03-19,Charlotte,127.000000,107.779239,3.0,3.0,0.0,15.0,54.0,25.0,6.0,3.0
45,2021-03-19,Denver,102.142857,86.347148,2.0,2.0,0.0,79.0,20.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
471,2022-09-09,San Francisco,65.000000,57.948063,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0
472,2022-09-09,NYC,2475.000000,2434.957070,2.0,2.0,0.0,51.0,42.0,6.0,1.0,2.0
473,2022-09-16,San Diego,95.571429,95.578798,2.0,2.0,12.0,87.0,1.0,0.0,0.0,2.0
474,2022-09-16,San Francisco,58.000000,59.520296,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0


## Hospital Rate Trend Categorization (HRT)

In [15]:
def HRT_categorize(realdata, prediction, threshold, predweek):

    # Converts continous predictions into HRT categorizations
    #
    # realdata: continous predictions
    # threshold: percentage threshold determining each level of categorization 
    # predweek: forecasting window

    rollavg = realdata.rolling(3).mean().shift(predweek) # identifies trend of previous 3 weeks
    GR = (prediction - rollavg)/rollavg # finds growth rate
    cats = []
    for value in GR:
        if np.isnan(value):
            cats.append(np.nan)
        elif value < -2* threshold:
            cats.append(-2)
        elif value < -threshold:
            cats.append(-1)
        elif value < threshold:
            cats.append(0)
        elif value < 2*threshold:
            cats.append(1)
        else:
            cats.append(2)
    return GR, cats

In [16]:
translate_HRT = {}
translate_HRT['date'] = "date"
translate_HRT['MLE_category'] = "MLE_category"
translate_HRT[2.0] = "Large Increase"
translate_HRT[1.0] = "Increase"
translate_HRT[0.0] = 'Stable'
translate_HRT[-1.0] = "Decrease"
translate_HRT[-2.0] = "Large Decrease"
 
def rs_HRT_categorize(realdata, randomsamples, threshold, predweek):
    # Converts random samples into HCR categorizations
    #
    # realdata: continous predictions
    # randomsamples: random samples
    # all_threshold: hospital capacity
    # predweek: forecasting window
    
    randsampcat = randomsamples[['date']].copy()
   
    for j in randomsamples.iloc[:,2:]:

        _, randsampcat[f'{j}'] = HRT_categorize(realdata, randomsamples[f"{j}"], threshold, predweek)

    randsampcat = randsampcat.set_index("date")

    randsampcat = randsampcat.apply(pd.Series.value_counts, axis=1).fillna(0).reset_index()
    randsampcat.index = thiscitysamp.index
    randsampcat['MLE_category'] = randsampcat.iloc[:,1:].idxmax(axis=1)
    randsampcat.columns = [translate_HRT[c] for c in randsampcat.columns]
    randsampcat
    
    return randsampcat


In [17]:
def brierscore(labels, probabilities, numcats):
    # Generates an array of 1 and 0 for label vs incorrect labels
    #
    # labels: correct labels
    # probailites: percent of random samples agreeing with each label
    # numcats: number of categories, depending 
    
    if numcats==5:
        labeled = [np.eye(numcats)[int(a+2)] for a in labels]
    elif numcats==3:
        labeled = [np.eye(numcats)[int(a+1)] for a in labels]
    
    # calculate mse of probability and labels and sum
    mses = np.sum(np.subtract(labeled, probabilities)**2, axis=1)

    # Average value shows how far off labels are on average
    return np.mean(mses)
                  
def brierskillscore(label, probabilties, numcats):
    # Compares brier score to random guess
    #
    # label: correct labels
    # probailites: percent of random samples agreeing with each label
    # numcats: number of categories, depending 
    
    
    bsreal = brierscore(label, probabilties, numcats)
    unformprob = 1/numcats * np.ones_like(probabilties) # equal probabilities for all labels 

    bsref = brierscore(label, unformprob, numcats)
    
    
    bss =  1 - (bsreal/bsref)
    return bss

### 1 Week Forecast

In [18]:
t1 = 0.1


In [19]:
catone_hrt = log_pred_df[['date','city_name', 'groundtruth1','1weekPred']].copy()
catone_hrt.columns=['date', 'city_name', 'hosp', 'predictions']
catone_hrt[['hosp_real_gr',"truth_category", "pred_category"]] = np.nan

for city in catone_hrt.city_name.unique():
    thiscity = catone_hrt[catone_hrt.city_name == city]
    thiscitysamp = RandSamp0.loc[thiscity.index]
    #realdata, prediction, threshold, predweek):
    growthrate_array, truthcat_array = HRT_categorize(thiscity.hosp, thiscity.hosp, t1, 1)
    _, predcat_array = HRT_categorize(thiscity.hosp, thiscity.predictions, t1, 1)
    catone_hrt.loc[thiscity.index, 'hosp_real_gr'] = growthrate_array
    catone_hrt.loc[thiscity.index, "truth_category"] = truthcat_array
    catone_hrt.loc[thiscity.index, "pred_category"] = predcat_array
    
    
    rs_ones = rs_HRT_categorize(thiscity.hosp, thiscitysamp, t1, 1)
    for category in rs_ones.columns[1:]:
        catone_hrt.loc[thiscitysamp.index, f'{category}'] = rs_ones[f'{category}']

catone_hrt[catone_hrt.pred_category.notnull()]

,date,city_name,hosp,predictions,hosp_real_gr,truth_category,pred_category,Large Decrease,Decrease,Stable,Increase,Large Increase,MLE_category
27,2021-02-26,San Francisco,100.833333,161.977536,-0.325780,-2.0,0.0,4.0,10.0,40.0,19.0,27.0,0.0
28,2021-02-26,NYC,6618.333333,7619.706645,-0.186787,-1.0,0.0,20.0,21.0,40.0,10.0,9.0,0.0
29,2021-02-26,Houston,884.142857,1196.610727,-0.159122,-1.0,1.0,2.0,7.0,33.0,20.0,38.0,2.0
30,2021-02-26,Denver,156.571429,151.563538,-0.012909,0.0,0.0,16.0,21.0,42.0,11.0,10.0,0.0
31,2021-03-05,Houston,796.857143,1033.782303,-0.162190,-1.0,0.0,2.0,9.0,42.0,22.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2022-09-09,San Francisco,76.000000,72.286986,-0.235327,-2.0,-2.0,83.0,15.0,2.0,0.0,0.0,-2.0
472,2022-09-09,NYC,2508.833333,2497.707282,-0.066519,0.0,0.0,7.0,30.0,58.0,4.0,1.0,0.0
473,2022-09-16,San Diego,89.261905,98.354995,-0.225733,-2.0,-1.0,26.0,44.0,29.0,1.0,0.0,-1.0
474,2022-09-16,San Francisco,71.333333,68.730117,-0.174807,-1.0,-2.0,52.0,37.0,11.0,0.0,0.0,-2.0


### 2 Week Forecast

In [20]:
cattwo_hrt = log_pred_df[['date','city_name', 'groundtruth2','2weekPred']].copy()
cattwo_hrt.columns=['date', 'city_name', 'hosp', 'predictions']
cattwo_hrt[['hosp_real_gr',"truth_category", "pred_category"]] = np.nan

for city in cattwo_hrt.city_name.unique():
    thiscity = cattwo_hrt[cattwo_hrt.city_name == city]
    thiscitysamp = RandSamp1.loc[thiscity.index]
    #realdata, prediction, threshold, predweek):
    growthrate_array, truthcat_array = HRT_categorize(thiscity.hosp, thiscity.hosp, t1, 2)
    _, predcat_array = HRT_categorize(thiscity.hosp, thiscity.predictions, t1, 2)
    cattwo_hrt.loc[thiscity.index, 'hosp_real_gr'] = growthrate_array
    cattwo_hrt.loc[thiscity.index, "truth_category"] = truthcat_array
    cattwo_hrt.loc[thiscity.index, "pred_category"] = predcat_array
    
    
    rs_twos = rs_HRT_categorize(thiscity.hosp, thiscitysamp, t1, 2)
    for category in rs_twos.columns[1:]:
        cattwo_hrt.loc[thiscitysamp.index, f'{category}'] = rs_twos[f'{category}']

cattwo_hrt[cattwo_hrt.pred_category.notnull()]


,date,city_name,hosp,predictions,hosp_real_gr,truth_category,pred_category,Large Decrease,Decrease,Stable,Increase,Large Increase,MLE_category
27,2021-02-26,San Francisco,79.500000,140.909480,-0.468425,-2.0,0.0,21.0,20.0,36.0,11.0,12.0,0.0
29,2021-02-26,Houston,796.857143,1102.504668,-0.242137,-2.0,0.0,10.0,13.0,36.0,15.0,26.0,0.0
30,2021-02-26,Denver,126.428571,147.058045,-0.202942,-2.0,0.0,25.0,19.0,35.0,9.0,12.0,0.0
31,2021-03-05,Houston,769.857143,946.755794,-0.190578,-1.0,0.0,12.0,17.0,41.0,14.0,16.0,0.0
32,2021-03-05,San Francisco,62.833333,97.648768,-0.498670,-2.0,-2.0,56.0,21.0,19.0,2.0,2.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2022-09-09,San Francisco,71.333333,63.962725,-0.282281,-2.0,-2.0,88.0,9.0,3.0,0.0,0.0,-2.0
472,2022-09-09,NYC,2451.500000,2444.049257,-0.087852,0.0,0.0,24.0,24.0,37.0,9.0,6.0,0.0
473,2022-09-16,San Diego,88.428571,96.389235,-0.232962,-2.0,-1.0,40.0,26.0,27.0,5.0,2.0,-2.0
474,2022-09-16,San Francisco,65.000000,63.867785,-0.248072,-2.0,-2.0,67.0,19.0,13.0,1.0,0.0,-2.0


### 3 week forecast

In [21]:
cattre_hrt = log_pred_df[['date','city_name', 'groundtruth3','3weekPred']].copy()
cattre_hrt.columns=['date', 'city_name', 'hosp', 'predictions']
cattre_hrt[['hosp_real_gr',"truth_category", "pred_category"]] = np.nan

for city in cattre_hrt.city_name.unique():
    thiscity = cattre_hrt[cattre_hrt.city_name == city]
    thiscitysamp = RandSamp2.loc[thiscity.index]
    #realdata, prediction, threshold, predweek):
    growthrate_array, truthcat_array = HRT_categorize(thiscity.hosp, thiscity.hosp, t1, 3)
    _, predcat_array = HRT_categorize(thiscity.hosp, thiscity.predictions, t1, 3)
    cattre_hrt.loc[thiscity.index, 'hosp_real_gr'] = growthrate_array
    cattre_hrt.loc[thiscity.index, "truth_category"] = truthcat_array
    cattre_hrt.loc[thiscity.index, "pred_category"] = predcat_array
    
    
    rs_tres = rs_HRT_categorize(thiscity.hosp, thiscitysamp, t1, 3)
    for category in rs_tres.columns[1:]:
        cattre_hrt.loc[thiscitysamp.index, f'{category}'] = rs_tres[f'{category}']

cattre_hrt[cattre_hrt.pred_category.notnull()]
cattre_hrt.tail()

,date,city_name,hosp,predictions,hosp_real_gr,truth_category,pred_category,Large Decrease,Decrease,Stable,Increase,Large Increase,MLE_category
471,2022-09-09,San Francisco,65.000000,57.948063,-0.346003,-2.0,-2.0,88.0,7.0,4.0,1.0,0.0,-2.0
472,2022-09-09,NYC,2475.000000,2434.957070,-0.079108,0.0,0.0,32.0,17.0,27.0,9.0,15.0,-2.0
473,2022-09-16,San Diego,95.571429,95.578798,-0.171004,-1.0,-1.0,45.0,17.0,23.0,6.0,9.0,-2.0
474,2022-09-16,San Francisco,58.000000,59.520296,-0.329049,-2.0,-2.0,71.0,13.0,12.0,2.0,2.0,-2.0
475,2022-09-16,NYC,2540.833333,2416.755388,-0.022067,0.0,0.0,29.0,16.0,28.0,10.0,17.0,-2.0


# Final Predictions

## HCR

In [22]:
catone_hcr.head()

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
31,2021-03-05,Houston,796.857143,1033.782303,4.0,5.0,0.0,0.0,0.0,9.0,91.0,5.0
32,2021-03-05,San Francisco,79.500000,116.962832,1.0,1.0,98.0,2.0,0.0,0.0,0.0,1.0
33,2021-03-05,Denver,126.428571,148.767082,2.0,2.0,0.0,69.0,30.0,1.0,0.0,2.0
34,2021-03-05,NYC,6042.000000,6788.576534,5.0,5.0,0.0,0.0,0.0,4.0,96.0,5.0
35,2021-03-05,Charlotte,199.333333,226.822857,3.0,4.0,0.0,0.0,21.0,54.0,25.0,4.0


In [23]:
cattwo_hcr.head()

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
36,2021-03-12,Houston,724.285714,811.598527,4.0,5.0,0.0,0.0,2.0,22.0,76.0,5.0
37,2021-03-12,NYC,5414.666667,5131.355208,5.0,5.0,0.0,0.0,2.0,16.0,82.0,5.0
38,2021-03-12,San Francisco,51.500000,64.839049,1.0,1.0,99.0,1.0,0.0,0.0,0.0,1.0
39,2021-03-12,Charlotte,149.666667,151.122482,3.0,4.0,0.0,4.0,44.0,40.0,12.0,3.0
40,2021-03-12,Denver,129.857143,66.690957,2.0,2.0,0.0,71.0,28.0,1.0,0.0,2.0


In [24]:
cattre_hcr.head()

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
41,2021-03-19,NYC,5246.166667,4779.088045,5.0,5.0,0.0,0.0,7.0,28.0,65.0,5.0
42,2021-03-19,San Francisco,38.000000,47.470068,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0
43,2021-03-19,Houston,616.714286,599.986608,4.0,5.0,0.0,0.0,7.0,33.0,60.0,5.0
44,2021-03-19,Charlotte,127.000000,107.779239,3.0,3.0,0.0,15.0,54.0,25.0,6.0,3.0
45,2021-03-19,Denver,102.142857,86.347148,2.0,2.0,0.0,79.0,20.0,1.0,0.0,2.0


## HRT

In [25]:
catone_hrt.head()

,date,city_name,hosp,predictions,hosp_real_gr,truth_category,pred_category,Large Decrease,Decrease,Stable,Increase,Large Increase,MLE_category
0,2021-01-08,Houston,1269.285714,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
1,2021-01-08,San Francisco,225.333333,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
2,2021-01-08,Denver,246.428571,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
3,2021-01-15,Denver,234.214286,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
4,2021-01-15,San Francisco,240.333333,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0


In [26]:
cattwo_hrt.head()

,date,city_name,hosp,predictions,hosp_real_gr,truth_category,pred_category,Large Decrease,Decrease,Stable,Increase,Large Increase,MLE_category
0,2021-01-08,Houston,1380.428571,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
1,2021-01-08,San Francisco,240.333333,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
2,2021-01-08,Denver,234.214286,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
3,2021-01-15,Denver,225.714286,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0
4,2021-01-15,San Francisco,227.666667,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-2.0


In [27]:
cattre_hcr.head()

,date,city_name,true_hosp,forecast_hosp,truth_category,pred_category,very_low,low,moderate,high,very_high,MLE_category
41,2021-03-19,NYC,5246.166667,4779.088045,5.0,5.0,0.0,0.0,7.0,28.0,65.0,5.0
42,2021-03-19,San Francisco,38.000000,47.470068,1.0,1.0,100.0,0.0,0.0,0.0,0.0,1.0
43,2021-03-19,Houston,616.714286,599.986608,4.0,5.0,0.0,0.0,7.0,33.0,60.0,5.0
44,2021-03-19,Charlotte,127.000000,107.779239,3.0,3.0,0.0,15.0,54.0,25.0,6.0,3.0
45,2021-03-19,Denver,102.142857,86.347148,2.0,2.0,0.0,79.0,20.0,1.0,0.0,2.0
